# 版权信息
## 许可证

本笔记本及其包含的所有内容（代码、文本、图片等），除非另有说明，均受MIT许可证保护：

- **作者**：[Xianglong KONG]
- **邮箱**：[kongxl@sdas.org]
- **创建日期**：[2023-06-01]
- **更新日期**：[2024-03-21]
- **版本**：[v0.5]

请在使用、分享或修改此笔记本时遵守上述许可证的条款。如果你对此笔记本有任何疑问或需要进一步的信息，请通过上面提供的邮箱与我联系。

In [ ]:
import os
import pandas as pd
# import lux
import numpy as np
import pathogenprofiler as pp
import json
import tbprofiler as tp
from collections import defaultdict
import sys
import argparse
import re
import csv
import subprocess


reverce={}
reverce['A'] = 'T'
reverce['T'] = 'A'
reverce['G'] = 'C'
reverce['C'] = 'G'

aa_long2short = {"Ala":"A","Arg":"R","Asn":"N","Asp":"D","Cys":"C","Gln":"Q","Glu":"E","Gly":"G","His":"H","Ile":"I","Leu":"L","Lys":"K","Met":"M","Phe":"F","Pro":"P","Ser":"S","Thr":"T","Trp":"W","Tyr":"Y","Val":"V","*":"*", 'fs':'fs',"?":"?"}
aa_short2long = {'A':'Ala', 'F':'Phe', 'C':'Cys', 'U':'Sec', 'D':'Asp', 'N':'Asn', 'E':'Glu', 'Q':'Gln', 'G':'Gly', 'H':'His', 'L':'Leu', 'I':'Ile', 'K':'Lys', 'O':'Pyl', 'M':'Met', 'P':'Pro', 'R':'Arg', 'S':'Ser', 'T':'Thr', 'V':'Val', 'W':'Trp', 'Y':'Tyr', '*':'STOP'}
codon2aa = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
    'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W'}

aa2codon = defaultdict(list)
for codon in codon2aa:
	aa2codon[codon2aa[codon]].append(codon)

class geneRecord():
	start = 0
	end = 0
	strand = 1
	description = ''
	gid = ''
	def __init__(self, gid, gname, start, end, strand, description):
		self.gid = gid
		self.gname = gname
		self.start = start
		self.end = end
		self.strand = strand
		self.description = description

def processGffGenes(fileName):
	genes = []
	with open(fileName) as fHandle:
		Crtype = 2
		Cgstart = 3
		Cgend = 4
		Cgstrand = 6
		Cgdescription = 8
		for line in fHandle:
			if line.startswith('#'):
				continue
			if line.startswith('##'):
				continue
			arr = line.strip().split()
			if len(arr) and arr[2].strip() == 'gene':
				gid = ""
				gname = ""
				for i in arr[Cgdescription].strip().split(';'):
					if "ID=" in i:
						gid = i.strip()[3:].replace("gene-Rv", 'Rv')
					if "Name=" in i:
						gname = i.strip()[5:]
				if arr[Cgstrand].strip() == '+':
					strand = 1
				else:
					strand = -1
				genes.append(geneRecord(gid, gname, int(arr[Cgstart].strip()), int(arr[Cgend].strip()), strand, arr[Cgdescription]))
	return genes

def getGene(genes, geneId) -> geneRecord:
	for gene in genes:
		if geneId in gene.gid:
			return gene
	return None

def getGenebyName(genes, gname):
	for gene in genes:
		if gname in gene.gname:
			return gene
	return None

def get_nt_change(gene_object,ref_aa, pos, mut_aa):
    ref_nt = ''
    pos_aa_to_nt = []
    pos_nt = 0
    mut_nt = ''

    g = gene_object
    if g != None and g.strand > 0:
        p = g.start + pos*3-3
        pos_aa_to_nt =  [p,p+1,p+2]
    elif g != None and g.strand < 0:
        p = g.end - pos*3 + 1
        pos_aa_to_nt = [p,p+1,p+2]       
    return pos_aa_to_nt

def get_nt_ref(ref_seq, gene_object, ref_aa, pos):
	ref_nt = ''
	pos_aa_to_nt = []
	g = gene_object
	if g != None and g.strand > 0:
		p = g.start + pos*3-3
		pos_aa_to_nt =  [p,p+1,p+2]
		ref_nt = ref_seq['AL123456.3'][p-1:p+2]
	elif g != None and g.strand < 0:
		p = g.end - pos*3
		pos_aa_to_nt = [p,p+1,p+2]     
		ref_nt = reverce[ref_seq['AL123456.3'][p+2]] + reverce[ref_seq['AL123456.3'][p+1]] + reverce[ref_seq['AL123456.3'][p]]
	return ref_nt

# Generate standardized SNP identifiers based on the position and VCF file.
# snpEff result process 
def gff_load_cds(gff):
    cds = []
    for l in open(gff):
        row = l.strip().split()
        if len(row)<3:
            continue
        if row[2]!="CDS":
            continue
        r = re.search("ID=(.+);+",l)
        id = r.group(1)
        cds.append({"chrom":row[0],"gene":id,"start":int(row[3]),"end":int(row[4]),"strand":row[6]})
    return cds

def get_overlapping_gene(chrom,pos,genes):
    genes = [g for g in genes if g["start"]<=pos and g["end"]>=pos and chrom==g["chrom"]]
    if len(genes)==0:
        return None
    else:
        return genes[0]

def get_codon_pos(chrom,pos,genes):
    g = get_overlapping_gene(chrom,pos,genes)
    if g==None:
        return (None,None)
    if g["strand"]=="+":
        codon_pos = (pos-g["start"])//3 + 1
    else:
        codon_pos = (g["end"] - pos )//3 + 1
    return (g["gene"],codon_pos)


ref_file = '/raid/slyy/pipelines/ref/GCA_000195955.2_ASM19595v2/GCA_000195955.2_ASM19595v2_genomic.fna'
gff_file = '/raid/slyy/pipelines/ref/GCA_000195955.2_ASM19595v2/GCA_000195955.2_ASM19595v2_genomic.gff'

ref_seq = pp.fasta(ref_file).fa_dict
gff = '/raid/slyy/pipelines/ref/GCA_000195955.2_ASM19595v2/GCA_000195955.2_ASM19595v2_genomic.gff'
genes = processGffGenes(gff)

ref = pp.fasta(ref_file).fa_dict
cds = gff_load_cds(gff_file)
coding = defaultdict(list)

samples_list_dir = '/slyy-backup/international-tb/data_tables/sample_lists/'

def write_list_to_file(l=[], fn = ''):
    with open(f'{samples_list_dir}{fn}', 'w') as fp:
        fp.write('\n'.join(l))
        
def read_list_from_file(fn=''):
    r = []
    with open(f'{samples_list_dir}{fn}', 'r') as fp:
        for line in fp:
            r.append(line.strip())
    return r

international_samples = read_list_from_file('international_samples.txt')

shandong_huada_samples = read_list_from_file('shandong_huada_1447.txt')

ancient_samples = ['ERR651004','ERR651000','ERR979066','ERR046946','ERR979064','ERR979065','ERR651003','ERR979067','ERR979063','ERR979062','ERR651001','ERR650975','ERR651010']

meta_file = '/slyy-backup/international-tb/data_tables/MTB_sra_international_samples_filtered_updated.csv'
masked_consensus_dir = '/slyy-backup/international-tb/output/snippy-masked-vcfs/'
output_dir_base = '/slyy-backup/international-tb/output/analysis/'
meta_df = pd.read_csv(meta_file)

shandong_sra_submission_file = '/slyy-backup/international-tb/data_tables/metadata-shandong-sra-submission.csv'
shandong_sra_submission_df = pd.read_csv(shandong_sra_submission_file)

# Read the data file using pandas.
mdr_snp_cluster_file = '/slyy-backup/international-tb/data_tables/PZA_stats.csv'   
mdr_snp_cluster = pd.read_csv(mdr_snp_cluster_file)

mdr_snp_cluster = mdr_snp_cluster[mdr_snp_cluster['IID'].isin(set(mdr_snp_cluster['IID']) & set(international_samples))]
mdr_snp_cluster = mdr_snp_cluster[mdr_snp_cluster['cluster2501'].notna()]
mdr_snp_cluster_l2 = mdr_snp_cluster[mdr_snp_cluster['IID'].isin(meta_df[meta_df['main_lin'] == 'lineage2']['sample_name'])]
mdr_snp_cluster_l4 = mdr_snp_cluster[mdr_snp_cluster['IID'].isin(meta_df[meta_df['main_lin'] == 'lineage4']['sample_name'])]

meta_df_mdr = meta_df[meta_df['sample_name'].isin(mdr_snp_cluster['IID'].to_list())]
meta_df_mdr = meta_df_mdr[meta_df_mdr['DR_type'].isin(['MDR-TB', 'Pre-XDR-TB'])]
meta_df_mdr_l2 = meta_df_mdr[meta_df_mdr['main_lin'] == 'lineage2']
meta_df_mdr_l4 = meta_df_mdr[meta_df_mdr['main_lin'] == 'lineage4']

liym_L221_20240107 = read_list_from_file('liym-l2.2.1-20240107.txt')
liym_L222_20240107 = read_list_from_file('liym-l2.2.2-20240107.txt')

# File retrieval function.
def findAllFile(base, ext):
    for root, ds, fs in os.walk(base):
        for f in fs:
            if f.endswith(ext):
                fullname = os.path.join(root, f)
                filename = os.path.splitext(f)[0]
                yield fullname, filename

def findAllDir(rootdir):
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            yield d, file

In [ ]:
# Convert Shandong IDs to SRA IDs in the International dataset.

dict_shandong_sra = dict(zip(shandong_sra_submission_df['sample_name'], shandong_sra_submission_df['accession']))
# meta_df[meta_df.sample_name.isin(shandong_sra_submission_df['sample_name'])]['sample_name'] = meta_df[meta_df.sample_name.isin(shandong_sra_submission_df['sample_name'])]['sample_name'].map(dict_shandong_sra)
meta_df.loc[meta_df.sample_name.isin(shandong_sra_submission_df['sample_name']), 'sample_name'] = meta_df[meta_df.sample_name.isin(shandong_sra_submission_df['sample_name'])]['sample_name'].map(dict_shandong_sra)
meta_df.to_csv('/slyy-backup/international-tb/data_tables/MTB_sra_international_samples_filtered_updated.csv')

In [ ]:
# Generate the TB-Profiler processing script file.

samples = set(meta_df.run_accession)

output_file = '/slyy-backup/international-tb/output/tb-profiler-620/tb620-runme.sh'
with open(output_file, 'w') as f:
    for s in samples:
        f.write(f'if [ ! -f results/{s}.results.json ]; then tb-profiler profile -1 /slyy-backup/international-tb/input-combine/{s}_1.fastq.gz -2 /slyy-backup/international-tb/input-combine/{s}_2.fastq.gz -p {s} --csv --call_whole_genome -t 10; fi;\n')

In [ ]:
import pandas as pd
import os
import numpy as np
import csv

# input: $ snp-dists -j 60 international.consensus.core2.fasta > international.vcf.consensus.core2.distance.tab

# Cluster based on the distance matrix.
# get clusters from distance matrix
def find_cluster(t):
    
    # Change the row and column names to sample names.

    samples = t.columns
    t.index = samples

    m = t
    result = []
    clusters = []
    unclustered = samples
    while(1):
        unclustered = list(set(samples) - set(result))
        s = unclustered
        m = t.loc[s,s]
        while (len(np.where(pd.isna(m))[0]) > 0):
            na_row = np.where(pd.isna(m))[0][0]

            indexes = m.loc[s[na_row],pd.notna(m.iloc[na_row])].index
            s = list(set(indexes))
            m = m.loc[s, s]
        if len(s) == 0: break
        if len(s) > 1: clusters.append(s)
        result = result + s
    return clusters

# Cluster the results based on SNP distance threshold and save the results as CSV.

def get_clusters(m, distance, result_dir,prefix='clusters-'): 
    md = m[m<=distance]
    clusters = find_cluster(md)
    with open(result_dir+prefix+str(distance)+'.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        header = "cluster", "sample"
        writer.writerow(header)
        for c,s in zip(range(len(clusters)), clusters):
            for i in range(len(s)):
                writer.writerow([c] + [s[i]])
            # t.loc[[s,s]].to_csv(result_dir+prefix+str(distance)+'.cluster'+str(c)+'.matrix'+'.csv')
        f.close()

In [ ]:
# Summarize the output results of samtools coverage.

import pandas as pd

bam_dir = '/slyy-backup/international-tb/output/tb-profiler-430-2/bam'

all_coverage = []
for i,j in findAllFile(bam_dir, '.coverage.out'):
    l = pd.read_csv(i, sep='\t')
    l['sample_name'] = j
    all_coverage.append(l)

df = pd.concat(all_coverage)
df.to_csv('/slyy-backup/international-tb/output/tb-profiler-430-2/coverage-stats.csv', sep=',', encoding='utf_8_sig')

NameError: name 'findAllFile' is not defined

In [ ]:
# Summarize the lineage information from TB Profiler results.
import json

result_dir = '/slyy-backup/international-tb/output/tb-combine/results'
all_lnn = []
for i,j in findAllFile(result_dir, '.json'):
        with open(i) as f:
            l = json.load(f)
            all_lnn.append(pd.DataFrame({'sample_name':j, 'main_lin':l['main_lin'], 'sub_lin':l['sublin']}, index = ['sample_name']))
df = pd.concat(all_lnn)

df.to_csv('/slyy-backup/international-tb/output/tb-combine/lineage_stats.csv')

In [ ]:
# Summarize the drug resistance information from TB Profiler results.

result_dir = '/slyy-backup/international-tb/output/tb-combine/results'
all_dr = []
for i,j in findAllFile(result_dir, '.json'):
        with open(i) as f:
            l = json.load(f)
            # print(l['dr_variants'])
            all_dr.append(pd.json_normalize(l, record_path='dr_variants'))

df = pd.concat(all_dr)
df.to_csv('/slyy-backup/international-tb/output/tb-combine/drug_res_stats.csv')

In [ ]:
from Bio import SeqIO
# Generate a merged sequence file using the recombined consensus sequences containing SNPs from the VCF file. Set the record ID as the sample name.

consensus_file_dir = '/slyy-backup/international-tb/output/snippy-vcfs-core2/'
ref_seq = '/raid/slyy/pipelines/ref/H37Rv.fasta'
records = []
for s in international_pza_dr_samples:
    if s == 'Reference':
        record = SeqIO.read(ref_seq, format='fasta')
        record.id = 'Reference'
        record.description = ''
        records.append(record)
    else:
        record = SeqIO.read(consensus_file_dir + s + '.core2.masked.snps.vcf.gz.fa', format='fasta')
        record.id = s
        record.description = ''
        records.append(record)
        
SeqIO.write(records, '/slyy-backup/international-tb/output/timetree/international.pza_1333.vcf.consensus.core2.fasta', format='fasta')


In [ ]:


def generate_input_aln_file_masked_consensus(sample_list=[], out_file='consensus.masked.fasta', out_dir=output_dir_base, in_dir=masked_consensus_dir):
    
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    records = []
    # Generate a merged sequence file using the recombined consensus sequences that contain SNPs from the VCF file. Set the record ID as the sample name.
    consensus_file_dir = masked_consensus_dir
    ref_seq = '/raid/slyy/pipelines/ref/H37Rv.fasta'

    record = SeqIO.read(ref_seq, format='fasta')
    record.id = 'Reference'
    records.append(record)

    for s in sample_list:
        record = SeqIO.read(in_dir + s + '.snps.subs.masked.vcf.gz.fa', format='fasta')
        record.id = s
        record.description = ''
        records.append(record)

    SeqIO.write(records, out_dir+out_file, format='fasta')
    

def iqtree_runner(seq_file='',outdir='',outfiles='',model='JC+I+G4', bs_reps='1000',alrt_reps='1000'):
    ## the following routine runs iqtree looking for the best model and tree, with no bootstrap. 
    ## It writes results to text files, but also returns the best fitting model.
    
    a = subprocess.run(['iqtree','-s',seq_file,'-m',model,'-bb',bs_reps,'-alrt',alrt_reps,'-pre',outdir+outfiles])
    print(a)
    
def treetime_runner(aln_file='', tree_file='', dates_file='', name_column='', date_column='', outdir='',clock_rate='4.684e-6'):
    a = subprocess.run(['treetime', '--aln', aln_file, '--tree', tree_file, '--dates', dates_file, '--name-column', name_column, '--date-column', date_column, 
                        '--outdir', outdir,
                        '--reroot', 'least-squares', '--reconstruct-tip-states', '--branch-length-mode', 'input', '--clock-filter', '3',
                        '--relax', '1.0', '0.5', '--clock-rate', clock_rate])
    print(a)
    
def iqtree_treetime_analysis(name='',samples=[]):
    generate_input_aln_file_masked_consensus(sample_list=samples, 
                                             out_file=name+'.consensus.masked.fasta', 
                                             out_dir=output_dir_base+name+'/')
    
    iqtree_runner(seq_file=output_dir_base+name+'/'+name+'.consensus.masked.fasta',
                  outdir=output_dir_base+name+'/',
                  outfiles=name+'.consensus.masked',
                  model='JC+I+G4', bs_reps='1000',alrt_reps='1000')
    
    treetime_runner(aln_file=output_dir_base+name+'/'+name+'.consensus.masked.fasta', 
                    tree_file=output_dir_base+name+'/'+name+'.consensus.masked.treefile', 
                    dates_file=meta_file, 
                    name_column='run_accession', 
                    date_column='collect_date', 
                    outdir=output_dir_base+name+'/',
                    clock_rate='4.684e-6')
    